<a href="https://colab.research.google.com/github/psychemistz/MultiDimGCNR/blob/main/FID_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing style-transfer using Frechet Inception Distance (FID)

https://wandb.ai/ayush-thakur/gan-evaluation/reports/How-to-Evaluate-GANs-using-Frechet-Inception-Distance-FID---Vmlldzo0MTAxOTI

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
drive_path = '/content/drive/MyDrive/COLAB_DRIVE/CycleGAN_Denoising_Code'

os.chdir(drive_path)
os.getcwd() 
!ls
###################################
import PIL.Image as Image

def image_normalize(filename):
  target_size = (256, 256)
  # images_arr = np.zeros((1, 256, 256, 3), dtype=np.float32)
  img = Image.open(filename)
  if (img.mode == 'CMYK')|(img.mode == 'RBFA'):
    img = img.convert('RGB')
  img = img.resize(target_size, Image.ANTIALIAS)
  if(len(np.asarray(img).shape)<3):
    img = np.expand_dims(np.asarray(img),axis=2)
    img = np.concatenate((img,img,img), axis=2)
  img = np.asarray(img).astype(np.float32)/255
  img = np.expand_dims(np.asarray(img),axis=0)
  return img
###################################################  
import tensorflow as tf
inception_model = tf.keras.applications.InceptionV3(include_top=False, 
                              weights="imagenet", 
                              pooling='avg')


###################################################  
from scipy import linalg
def calculate_fid(real_embeddings, generated_embeddings):
  # calculate mean and covariance statistics
  mu1, sigma1 = real_embeddings.mean(axis=0), np.cov(real_embeddings, rowvar=False)
  mu2, sigma2 = generated_embeddings.mean(axis=0), np.cov(generated_embeddings,  rowvar=False)
  # calculate sum squared difference between means
  ssdiff = np.sum((mu1 - mu2)**2.0)
  # calculate sqrt of product between cov
  covmean = linalg.sqrtm(sigma1.dot(sigma2))
  # check and correct imaginary numbers from sqrt
  if np.iscomplexobj(covmean):
    covmean = covmean.real
  # calculate score
  fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

Mounted at /content/drive
architecture		       progress
Calculate_GMDM_color_wise.asv  __pycache__
Calculate_GMDM_color_wise.m    style_analysis_figure.fig
Calculate_GMDM.m	       style_analysis_figure.png
CycleGAN_Training.py	       style_analysis_results.mat
data			       summaries
GSSMD.m			       Test.py
kernelpca_tutorial.m	       training_weights
load_images.m		       utils.py
pca_proj.m		       Visualize_Distribution_Shift.m
87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
checkpoints = ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000']
fid_inp_tar =[]
fid_inp_out =[]
fid_out_tar =[]

for chkpt in checkpoints:
# eval_checkpoint = './data/Test/predicted_1500'
  eval_checkpoint = './data/Test/predicted_' + chkpt
  input_path   = eval_checkpoint + '/input_img'
  label_path   = eval_checkpoint + '/target_img'
  output_path   = eval_checkpoint + '/output_img'

  import glob
  import numpy as np
  input_list = glob.glob(input_path+"*.jpg")
  target_list = glob.glob(label_path+"*.jpg")
  output_list = glob.glob(output_path+"*.jpg")

  num_samples    = len(input_list)
  counter = 0

  input_embeddings = []
  target_embeddings = []
  output_embeddings = []

  for sam_loop in range(num_samples):
  # for sam_loop in range(3):
        input_path = input_list[sam_loop]
        target_path = target_list[sam_loop]
        output_path = output_list[sam_loop]
        input_embeddings.extend(inception_model.predict(image_normalize(input_path)))
        target_embeddings.extend(inception_model.predict(image_normalize(target_path)))
        output_embeddings.extend(inception_model.predict(image_normalize(output_path)))

  input_embeddings = np.array(input_embeddings)
  target_embeddings = np.array(target_embeddings)
  output_embeddings = np.array(output_embeddings)

  fid_inp_tar.append(calculate_fid(input_embeddings, target_embeddings))
  fid_inp_out.append(calculate_fid(input_embeddings, output_embeddings))
  fid_out_tar.append(calculate_fid(output_embeddings, target_embeddings))

  print('FID-scores for the checkpoint: ',chkpt)
  print('fid_inp_tar: ', fid_inp_tar[-1])
  print('fid_inp_out: ', fid_inp_out[-1])
  print('fid_out_tar: ', fid_out_tar[-1])

fid_inp_tar = np.array(fid_inp_tar)
fid_inp_out = np.array(fid_inp_out)
fid_out_tar = np.array(fid_out_tar)

FID-scores for the checkpoint:  500
fid_inp_tar:  168.83003253726872
fid_inp_out:  258.26930478402477
fid_out_tar:  308.296510317429
FID-scores for the checkpoint:  1000
fid_inp_tar:  168.83003260889706
fid_inp_out:  256.26481634843054
fid_out_tar:  296.05906403179387
FID-scores for the checkpoint:  1500
fid_inp_tar:  168.8300324058848
fid_inp_out:  295.5416917759486
fid_out_tar:  292.5013375281662
FID-scores for the checkpoint:  2000
fid_inp_tar:  168.83003251929364
fid_inp_out:  275.18955774967446
fid_out_tar:  273.30281544350566
FID-scores for the checkpoint:  2500
fid_inp_tar:  168.83003251929364
fid_inp_out:  218.54357978084755
fid_out_tar:  219.63597557599016
FID-scores for the checkpoint:  3000
fid_inp_tar:  168.83003251929364
fid_inp_out:  230.8240472053322
fid_out_tar:  226.522456596332
FID-scores for the checkpoint:  3500
fid_inp_tar:  168.83003251929364
fid_inp_out:  240.84291303705126
fid_out_tar:  236.6427732793754
FID-scores for the checkpoint:  4000
fid_inp_tar:  168.830

In [ ]:
# FID-scores for the checkpoint: 500
# fid_inp_tar:  168.83003388175507
# fid_inp_out:  258.2693118067482
# fid_out_tar:  308.2964975718319

# FID-scores for the checkpoint: 1000
# fid_inp_tar:  168.8300338335796
# fid_inp_out:  256.2648310046467
# fid_out_tar:  296.059060058215

# FID-scores for the checkpoint: 1500
# fid_inp_tar:  168.83003379220037
# fid_inp_out:  295.54172450003034
# fid_out_tar:  292.5013472703527

# FID-scores for the checkpoint: 2000
# fid_inp_tar:  168.83003372699216
# fid_inp_out:  275.1896055159737
# fid_out_tar:  273.3028210959583

# FID-scores for the checkpoint: 2500  ---> THIS
# fid_inp_tar:  168.83003372699216
# fid_inp_out:  218.54361522643669
# fid_out_tar:  219.63599119793108

# FID-scores for the checkpoint: 3000
# fid_inp_tar:  168.83003372699216
# fid_inp_out:  230.82405727459278
# fid_out_tar:  226.52246160064112

# FID-scores for the checkpoint: 3500
# fid_inp_tar:  168.83003372699216
# fid_inp_out:  240.84293241872973
# fid_out_tar:  236.6427687144859

# FID-scores for the checkpoint: 4000 ---> and THIS
# fid_inp_tar:  168.83003372699216
# fid_inp_out:  210.7570881122477
# fid_out_tar:  219.56681681381409